In [1]:
import pandas as pd
import pyreadr

In [2]:
data = pyreadr.read_r('r_data/mydata_af.0.r')

In [10]:
data['mydata_af.0']

,gvkey,datadate,indfmt,datafmt,consol,popsrc,acchg,accli,acco,accrt,...,xstfo,xstfws,xt,xuw,xuwli,xuwnli,xuwoi,xuwrei,xuwti,PacVerToFeedPop2
0,1083.0,2.866176e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5407.0
1,1139.0,2.866176e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5407.0
2,1173.0,2.866176e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5407.0
3,1240.0,2.866176e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5407.0
4,1301.0,2.866176e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5407.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261597,162906.0,1.575101e+09,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5407.0
261598,171111.0,1.575101e+09,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5407.0
261599,183287.0,1.575101e+09,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5407.0
261600,183739.0,1.575101e+09,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5407.0
